In [1]:
!pip install google-cloud-speech
!pip install google-cloud-storage
!pip install pydub
!pip install https://github.com/pyannote/pyannote-audio/archive/develop.zip
!pip install speechbrain

  Using cached https://github.com/pyannote/pyannote-audio/archive/develop.zip (13.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\minhl\AppData\Local\Temp\pip-req-build-k9kjvqs_\setup.py", line 9, in <module>
          long_description = f.read()
        File "C:\Users\minhl\anaconda3\envs\s2t_GCP_Final\lib\encodings\cp1252.py", line 23, in decode
          return codecs.charmap_decode(input,self.errors,decoding_table)[0]
      UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 3425: character maps to <undefined>
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned abo

In [2]:
## Library
import os
from pydub import AudioSegment, silence
import wave
from pyannote.audio import Pipeline
from google.cloud import speech
from google.cloud import storage

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Speech2Text.json'

## Tạo thư mục cho từng tác vụ
Thu_muc_cho_audio_cat_theo_giong_noi = 'Splited_speaker'
Thu_muc_cho_audio_cat_theo_silence   = 'Splited_Silence'

os.makedirs(Thu_muc_cho_audio_cat_theo_giong_noi ,exist_ok= True)
os.makedirs(Thu_muc_cho_audio_cat_theo_silence   ,exist_ok= True)


c:\Users\minhl\anaconda3\envs\s2t_GCP_Final\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 505/505 [00:00<00:00, 505kB/s]
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
Downloading: 100%|██████████| 17.7M/17.7M [00:01<00:00, 10.7MB/s]
Downloading: 100%|██████████| 1.98k/1.98k [00:00<00:00, 1.99MB/s]


In [3]:
# Split Speaker
def Split_speaker(audio_file):
    Audio = AudioSegment.from_file(audio_file)
    name = Thu_muc_cho_audio_cat_theo_giong_noi + '/' + audio_file.split('/')[-1].replace('.wav', '')

    os.makedirs(name, exist_ok=True)
    hashDict = {}
    diarization = pipeline(audio_file)
    
    i,start,end,lab = 0,0,0,0
    for segment,track,label in diarization.itertracks(yield_label = True):
        if label != lab:
            Audio_speaker_i = Audio[start * 1000 : end * 1000]
            Audio_speaker_i.export(name + '/' + str(i) + '.wav', format = 'wav')
            hashDict[name + '/' +str(i) + '.wav'] = lab
            i += 1
            start = segment.start
            end = segment.end
            lab = label
        elif label == lab:
            end = segment.end
    # chạy vòng cuối
    Audio_speaker_i = Audio[start * 1000 : end * 1000]
    Audio_speaker_i.export(name +'/' + str(i) + '.wav', format = 'wav')
    hashDict[name + '/' +str(i) + '.wav'] = lab
    
    os.remove(name + '/' + str(0) + '.wav')
    hashDict.pop(name + '/' +str(0) + '.wav')
    return hashDict

In [4]:
# Split Silence
def export_audio(audio, count,name):
    audios = audio.set_frame_rate(16000)
    audios.export(os.path.join(name + '/file_{}.wav'.format(str(count))), format='wav')

# hashDict: {speaker: (start, stop)}
def split_silence(audio, speaker):
    folder = Thu_muc_cho_audio_cat_theo_silence
    os.makedirs(folder, exist_ok=True)
    name = audio.split('/')
    name = name[-2] + "/" + name[-1].replace('.wav','')
    name = os.path.join(folder,name)
    os.makedirs(name, exist_ok=True)

    myaudio = AudioSegment.from_file(audio, "wav")
    dbfs = myaudio.dBFS
    duration_in_sec = len(myaudio) / 1000

    mydict = {}
    t_dict = {}

    # Lấy các khoảng silence trong audio
    silences = silence.detect_silence(myaudio,
                                      min_silence_len= 300,
                                      silence_thresh=dbfs-10)
    silences = [((start/1000),(stop/1000)) for start,stop in silences]
    # print(silences)

    if len(silences) > 0:
        n_silence = []
        if silences[0][0] == 0.0:
            n_silence.append(silences[0])
            silences.pop(0)

        # Chỉnh lại, làm tròn sec
        for i in silences:
            if round(i[0]) < i[0]:
                temp= (i[0]+0.5, i[1])
            else:
                temp= (round(i[0]), i[1])
            n_silence.append(temp)
        # print(n_silence)

        count = 1
        for start,end in silences:
            temp= name+'/file_'+str(count)+'.wav' # vị trí file: lưu file ở thư mục nào thì địa chỉ tới thư mục đó
            t_dict[temp]= end-start
            count +=1
        

        # export_file_audio
        start = 0.0
        end = duration_in_sec
        s_audio = myaudio[start*1000:n_silence[0][0]*1000]
        export_audio(s_audio, 1,name)
        count = 2
        for i in range(len(n_silence)-1):
            s= n_silence[i][1]
            e= n_silence[i+1][0]
            n_audio = myaudio[s*1000:e*1000]
            export_audio(n_audio, count,name)
            count += 1
        if n_silence[len(n_silence)-1][1] != end:
            e_audio = myaudio[n_silence[len(n_silence)-1][1]*1000:end*1000]
            export_audio(e_audio, count,name)
            temp= name + '/file_'+str(count)+'.wav'
            t_dict[temp] = 0
    else:
        temp= name+'/file_'+str(0)+'.wav'
        export_audio(myaudio, 0,name)
        t_dict[temp] = 0
    # final
    mydict[speaker] = t_dict
    return mydict
    # return: {speaker: {file_split_1: (silence_time)}, {file_split_2: (silence_time)}, ...}

In [5]:
# tổng hợp bucket hiện có
def list_buckets():
    """Lists all buckets."""

    storage_client = storage.Client()
    buckets = storage_client.list_buckets()

    for bucket in buckets:
        print(bucket.name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [6]:
# Một số Function hỗ trợ xử lý văn bản đầu ra
def is_digit(word):
    try:
        int(word)
        return True
    except ValueError:
        pass
    return False


def ConvertDate(text):
    month=' tháng '
    year=' năm '
    for index in range(0,len(text)):
        try:
            if (text.index(month,index)==index):
                dateNum = text[index -1]
                monthNum = text[index + len(month)]
                if is_digit(dateNum) and is_digit(monthNum):
                    text=text[:index] + text[index+len(month)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
        try:
            if (text.index(year,index)==index):
                monthNum = text[index -1]
                yearNum = text[index + len(year)]
                if is_digit(monthNum) and is_digit(yearNum):
                    text=text[:index] + text[index+len(year)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
    return text

In [7]:
# Các Function xử dụng GCP API thực hiện speech to text
def frame_rate_channel(audio_file_name):
    print(audio_file_name)
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

## Config_GGC_model_before_Transcribe
def Config_GGC(sample_rate_hertz = 44100,
                audio_channel_count = 1,
                model = None,
                enable_automatic_punctuation=True):
    if model != None:
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=enable_automatic_punctuation,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
            model = model,
        )
    else :
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            enable_automatic_punctuation=True,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced
## Config_GGC that doesn't have punctuation
def Config_noPunc(sample_rate_hertz = 44100,
                audio_channel_count = 1,
                model = None,
                enable_automatic_punctuation=True):
    if model != None:
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
            model = model,
        )
    else :
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count
        )
    return config_wav_enhanced

def Transcribe_Short_Audio(Audio_wav,config_wav_enhanced):
    client = speech.SpeechClient()
    with io.open(Audio_wav, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    # print(type(audio))
    
    response = client.recognize(config=config_wav_enhanced, audio=audio)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        alter=ConvertDate(alternative.transcript+ '\n')
#         print("-" * 20)
#         print("First alternative of result {}".format(i))
#         print("Transcript: {}".format(alternative.transcript))
        text.append(alter)
    return text

### This function is main on stranscribe
def Transcribe_Long_Audio(Audio_wav,config_wav_enhanced,
                        bucket_name = 'speech_to_text_stech',
                        Name = 'Audio_wav'):
    client = speech.SpeechClient()
    Audio_name = Audio_wav.split('/')[-1].split('.')[0]
    
    upload_blob(bucket_name,Audio_wav,Name)
    
    media_uri = "gs://{}/{}".format(bucket_name,Name)
    long_audi_wav = speech.RecognitionAudio(uri=media_uri)
    
    
    operations = client.long_running_recognize(
        config = config_wav_enhanced,
        audio = long_audi_wav
    )
    
    response = operations.result(timeout=90)

    text = []
    
    for i, result in enumerate(response.results):
            alternative = result.alternatives[0]
            alter=ConvertDate(alternative.transcript)
#             print("-" * 20)
#             print("First alternative of result {}".format(i))
#             print("Transcript: {}".format(alternative.transcript))
            text.append(alter)
    return text

In [8]:
##Take audio to text+punc
def AudioToText(path):
    # Audio_wav="../Example/12_no_music.wav"
    #Audio_wav=LoadAudio()
    rate,channel=frame_rate_channel(path)
    config = Config_GGC(sample_rate_hertz = rate,
                 audio_channel_count = channel)
    text = Transcribe_Long_Audio(path,config)
    return text

##Take audio to text, but no punc
def AudioToNoPuncText(path):
    rate,channel=frame_rate_channel(path)
    config_nopunc = Config_noPunc(sample_rate_hertz = rate,
                audio_channel_count = channel)
    text_nopunc = Transcribe_Long_Audio(path,config_nopunc)
    return text_nopunc

In [9]:
# Sử dụng Google API để dịch từng Audio sang text
def GCP_s2t(alist):
    result_dict = {'Speaker':[], 'co dau':[],'khong dau':[],'silence':[]}
    for adict in alist:
        for speaker,value in adict.items():
            for file_addr,silent_time in adict[speaker].items():
                
                text=AudioToText(file_addr)
                text_noPunc=AudioToNoPuncText(file_addr)

                result_dict['Speaker'].append(speaker)
                result_dict['co dau'].append(text)
                result_dict['khong dau'].append(text_noPunc)
                result_dict['silence'].append(silent_time)
    return result_dict

In [10]:
# Function Kết hợp Text có dấu theo Google API và silence

def Handle_Text_Dict(adict,S_Punc_ok=False):
    adict = list(adict.values())
    Spr,SL = 0,0
    ReDict = {'Speaker':[], 'co dau':[],'khong dau':[]}
    Silent_Punc = []
    for Speaker,Punc,NoPunc,Silent in zip(*adict):
        # Vì Punc,NoPunc là list
        new_Punc = ''
        for i in Punc:
            new_Punc += i
        new_NoPunc = ''
        for i in NoPunc:
            new_NoPunc += i
        
        if Speaker != Spr:
            # Làm gì đó
            Spr = Speaker
            SL = Silent
            ReDict['Speaker'].append(Speaker)
            ReDict['co dau'].append(new_Punc)
            ReDict['khong dau'].append(new_NoPunc)
            if S_Punc_ok == True:
                Silent_Punc.append(new_NoPunc)
        else: 
            # print(ReDict['khong dau'])
            ReDict['khong dau'][-1] += ' ' + new_NoPunc
            ReDict['co dau'][-1] += ' ' + new_Punc
            # # Xét Silence để thêm dấu
            # if S_Punc_ok == True:
            #     if Silent < Giới_hạn:
            #         Silent_Punc[-1] += ', ' + new_NoPunc
            #     else
            #         Silent_Punc[-1] += '. ' + new_NoPunc
    return ReDict,Silent_Punc



In [11]:
# Phần ghép các Function
Audio_Input = 'Input/23-06-2022 14 12 36.wav'

# Đưa các lệnh dưới đây của cell này vô Function là ta có Pipeline

# splited speaker
hashDict = Split_speaker(Audio_Input)

# splited Silence
My_info = []
for audio,speaker in hashDict.items():
    my_dict = split_silence(audio,speaker)
    My_info.append(my_dict)
print(My_info)

# Chuyển từng audio thành text có dấu và không dấu
Text_Dict=GCP_s2t(My_info)

# Chuyển thành Dict theo yêu cầu của Tín và để cho phần tìm silent
Result,_ = Handle_Text_Dict(Text_Dict)


[{'SPEAKER_01': {'Splited_Silence\\23-06-2022 14 12 36/1/file_0.wav': 0}}, {'SPEAKER_02': {'Splited_Silence\\23-06-2022 14 12 36/2/file_1.wav': 0.351, 'Splited_Silence\\23-06-2022 14 12 36/2/file_2.wav': 0.36299999999999955, 'Splited_Silence\\23-06-2022 14 12 36/2/file_3.wav': 0.4380000000000024, 'Splited_Silence\\23-06-2022 14 12 36/2/file_4.wav': 0.5449999999999982, 'Splited_Silence\\23-06-2022 14 12 36/2/file_5.wav': 0.4499999999999993, 'Splited_Silence\\23-06-2022 14 12 36/2/file_6.wav': 0.7989999999999995, 'Splited_Silence\\23-06-2022 14 12 36/2/file_7.wav': 0}}, {'SPEAKER_01': {'Splited_Silence\\23-06-2022 14 12 36/3/file_0.wav': 0}}, {'SPEAKER_00': {'Splited_Silence\\23-06-2022 14 12 36/4/file_1.wav': 0.32500000000000107, 'Splited_Silence\\23-06-2022 14 12 36/4/file_2.wav': 0.4980000000000011, 'Splited_Silence\\23-06-2022 14 12 36/4/file_3.wav': 0.31200000000000117, 'Splited_Silence\\23-06-2022 14 12 36/4/file_4.wav': 0.3940000000000019, 'Splited_Silence\\23-06-2022 14 12 36/4/f

In [16]:
print(type(My_info))
print(len(My_info))
print(My_info[1])

<class 'list'>
5
{'SPEAKER_02': {'Splited_Silence\\23-06-2022 14 12 36/2/file_1.wav': 0.351, 'Splited_Silence\\23-06-2022 14 12 36/2/file_2.wav': 0.36299999999999955, 'Splited_Silence\\23-06-2022 14 12 36/2/file_3.wav': 0.4380000000000024, 'Splited_Silence\\23-06-2022 14 12 36/2/file_4.wav': 0.5449999999999982, 'Splited_Silence\\23-06-2022 14 12 36/2/file_5.wav': 0.4499999999999993, 'Splited_Silence\\23-06-2022 14 12 36/2/file_6.wav': 0.7989999999999995, 'Splited_Silence\\23-06-2022 14 12 36/2/file_7.wav': 0}}


In [12]:
print(Text_Dict.keys())
print('\nText_Dict[\'Speaker\']  :',Text_Dict['Speaker'][0:5])
print('\nText_Dict[\'co dau\']   :',Text_Dict['co dau'][0:5])
print('\nText_Dict[\'khong dau\']:',Text_Dict['khong dau'][0:5])
print('\nText_Dict[\'silence\']  :',Text_Dict['silence'][0:5])
### Dict lúc sau

a=[{'speaker1': {'file_addr1': 0.5 , # silence_time
                 'file_addr2':  0.6}},
    {'speaker2': {'file_addr1': 0.5 ,
                  'file_addr2':  0.6}}
    ]

dict_keys(['Speaker', 'co dau', 'khong dau', 'silence'])

Text_Dict['Speaker']  : ['SPEAKER_01', 'SPEAKER_02', 'SPEAKER_02', 'SPEAKER_02', 'SPEAKER_02']

Text_Dict['co dau']   : [['Xin mời đồng chí Vũ Thanh'], ['Ngày 13/6, Bộ Chỉ huy Bộ đội biên phòng Bà Rịa Vũng Tàu cho biết qua hai ngày 11 và 12/6.'], ['kế hoạch tuần tra, bảo vệ chủ quyền vùng biển và phối hợp bảo vệ an ninh an toàn đường ống dẫn khí Nam Côn Sơn'], ['Lực lượng Biên phòng và công ty đường ống khí Nam Côn Sơn đã phát hiện 12 Vụ việc với 17 Phương tiện đang nghe.'], ['Khai thác đánh bắt thủy hải sản và cận kề hành lang an toàn đường ống dẫn khí dưới biển.']]

Text_Dict['khong dau']: [['Xin mời đồng chí Vũ Thanh'], ['ngày 13/6 Bộ Chỉ huy Bộ đội biên phòng Bà Rịa Vũng Tàu cho biết qua hai ngày 11 và 12/6'], ['kế hoạch tuần tra bảo vệ chủ quyền vùng biển và phối hợp bảo vệ an ninh an toàn đường ống dẫn khí Nam Côn Sơn'], ['lực lượng Biên phòng và công ty đường ống khí Nam Côn Sơn đã phát hiện 12 Vụ việc với 17 Phương tiện 

In [13]:
print(Result.keys())
print('\nResult[\'Speaker\']  :',Result['Speaker'][0:5])
print('\nResult[\'co dau\']   :',Result['co dau'][0:5])
print('\nResult[\'khong dau\']:',Result['khong dau'][0:5])

dict_keys(['Speaker', 'co dau', 'khong dau'])

Result['Speaker']  : ['SPEAKER_01', 'SPEAKER_02', 'SPEAKER_01', 'SPEAKER_00', 'SPEAKER_01']

Result['co dau']   : ['Xin mời đồng chí Vũ Thanh', 'Ngày 13/6, Bộ Chỉ huy Bộ đội biên phòng Bà Rịa Vũng Tàu cho biết qua hai ngày 11 và 12/6. kế hoạch tuần tra, bảo vệ chủ quyền vùng biển và phối hợp bảo vệ an ninh an toàn đường ống dẫn khí Nam Côn Sơn Lực lượng Biên phòng và công ty đường ống khí Nam Côn Sơn đã phát hiện 12 Vụ việc với 17 Phương tiện đang nghe. Khai thác đánh bắt thủy hải sản và cận kề hành lang an toàn đường ống dẫn khí dưới biển. Ngày đó tổ tuần tra tàu biên phòng 1398 01 thuộc Hải đội biên phòng 2. Bộ đội biên phòng Bà Rịa Vũng Tàu đã tuần tra dọc hành lang an toàn đường ống dẫn khí Nam Côn Sơn từ km 22 km. B75 và ngược lại bắt cá.', 'Xin mời đồng chí Trương Thị Lan', 'Quá trình tuần tra tổ công tác phát hiện 12 Vụ việc với 17 Phương tiện đang Neo độ khai thác đánh bắt thủy sản trong và cận kề hành lang an toàn đường ống dẫn kh